In [ ]:
import pandas as pd
import geopandas as gpd

import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as pyo

# Set notebook mode to work in offline
pyo.init_notebook_mode()

In [ ]:
# Utility function that creates Plotly choropleth map from geodataframe
def build_plotly_map(dataframe: gpd.GeoDataFrame, unit_name_column: str) -> go.Figure:
    # Extract the bounding box of the geometries
    bounds = dataframe.total_bounds

    # Create the Plotly figure
    fig = px.choropleth(dataframe,
                        geojson=dataframe.geometry,
                        locations=dataframe.index,
                        color='value',
                        color_continuous_scale='Viridis'
                       )

    fig.update_traces(
        hovertemplate="<br>".join([
            "Geographic unit<br><b>%{customdata[0]}</b>",
            "Value<br><b>%{customdata[1]}</b>"
        ]),
        customdata=dataframe[[unit_name_column, 'value']].values
    )

    # Update the layout to zoom to the bounding box
    fig.update_geos(
        visible=False,
        resolution=110,
        showcountries=True,
        countrycolor="#212721",
        showsubunits=True,
        showcoastlines=True,
        projection_type="mercator",
        fitbounds="locations",
        center={"lat": (bounds[1] + bounds[3]) / 2, "lon": (bounds[0] + bounds[2]) / 2},
        lonaxis_range=[bounds[0], bounds[2]],
        lataxis_range=[bounds[1], bounds[3]]
    )
    return fig


In [ ]:
# Get Crop data points and convert into the dataframe

# Paste URL from https://fdw.fews.net/data-explorer/crop
crop_url = "https://fdw.fews.net/api/cropproductionfacts/?format=json&has_datapoints=true&cpcv2=R01592AA&indicator=crop:quantity&country_code=NE&fields=simple"

crop_dataframe = pd.read_json(crop_url)
crop_dataframe.head()


In [ ]:
crop_dataframe.info()

In [ ]:
# Get spatial boundaries and convert into the geodataframe

# Paste URL from https://fdw.fews.net/data-explorer/spatial
spatial_url = "https://fdw.fews.net/api/feature/?format=geojson&unit_set_version=244&unit_set_version=268&unit_set_version=8&unit_set_version=290&unit_set_version=813"

spatial_dataframe = gpd.read_file(spatial_url)
spatial_dataframe.head()


In [ ]:
spatial_dataframe.info()

In [ ]:
# Add spatial info to the crop data points by merging two dataframes

dataframe = spatial_dataframe[['fnid', 'unit_type', 'full_name', 'geometry']].merge(
    crop_dataframe, on='fnid', how='right'
)
dataframe.info()

In [ ]:
# Filter dataframe to get data for a single year and administrative unit type

period_date = '2019-09-30'
unit_type = 'Admin 2'

single_year_dataframe = dataframe[
    (dataframe['period_date'] == period_date) & (dataframe['unit_type'] == unit_type) & (dataframe['status'] == 'Collected')
].reset_index(drop=True)

single_year_dataframe[['fnid', 'full_name', 'unit_type', 'period_date', 'value', 'indicator']]


In [ ]:
# Create a map for a single year and unit type

build_plotly_map(single_year_dataframe, 'full_name')

In [ ]:
# Get data points with geometries and create a map from a single API call

crop_spatial_url = "https://fdw.fews.net/api/cropproductionfacts/?format=geojson&has_datapoints=true&cpcv2=R01592AA&indicator=crop:quantity&country_code=NE&period_date=2019-09-30&geographic_unit_type=admin2"
crop_spatial_dataframe = gpd.read_file(crop_spatial_url).dropna(subset=['value']).reset_index(drop=True)
crop_spatial_dataframe[['fnid', 'geographic_unit_full_name', 'period_date', 'value', 'indicator']]


In [ ]:
crop_spatial_dataframe.info()

In [ ]:
build_plotly_map(crop_spatial_dataframe, 'geographic_unit_full_name')